# Testing 

In [3]:
!nvidia-smi

Fri May 30 20:16:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.70                 Driver Version: 572.70         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4080      WDDM  |   00000000:01:00.0  On |                  N/A |
|  0%   44C    P8              6W /  320W |    1548MiB /  16376MiB |      2%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
from diffusers import StableDiffusionImg2ImgPipeline
import torch, argparse, pathlib, random, PIL.Image as Image

def main(inp, out):
    pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5",
        torch_dtype=torch.float16       # use fp16 on your 4080
    ).to("cuda")

    prompts = pathlib.Path("prompts.txt").read_text().strip().splitlines()
    prompt  = random.choice(prompts)

    init = Image.open(inp).convert("RGB").resize((512, 512))
    image = pipe(
        prompt,
        image=init,
        strength=0.65,          # 0 = keep photo, 1 = overwrite it
        guidance_scale=7.5      # “creativity slider”
    ).images[0]

    pathlib.Path(out).parent.mkdir(parents=True, exist_ok=True)
    image.save(out)
    print(f"✨  saved → {out}")